In [34]:
import pandas as pd

In [35]:
# Load the datasets
df_ci = pd.read_csv('customer_interactions.csv')
df_pd = pd.read_csv('product_details.csv', delimiter=',')
df_ph = pd.read_csv('purchase_history.csv', delimiter=',')

In [36]:
df_merge = pd.merge(df_ph, df_ci, on='customer_id', how='left')
df = pd.merge(df_merge, df_pd, on='product_id', how='left')

In [37]:
# Feature Engineering
# Customer Behavior Features
df['total_page_views'] = df.groupby('customer_id')['page_views'].transform('sum')
df['average_time_spent'] = df.groupby('customer_id')['time_spent'].transform('mean')

# Purchase History Features
df['total_purchases'] = df.groupby('customer_id')['product_id'].transform('nunique')
df['recency_of_purchase'] = (pd.to_datetime('now') - pd.to_datetime(df['purchase_date'])).dt.days

# Product Interaction Features
df['average_rating_purchased'] = df.groupby('customer_id')['ratings'].transform('mean')
df['average_price_purchased'] = df.groupby('customer_id')['price'].transform('mean')

# Temporal Features
df['purchase_month'] = pd.to_datetime(df['purchase_date']).dt.month

C:\Users\ASUS\anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:2199: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  result, tz_parsed = tslib.array_to_datetime(


In [38]:
df

,customer_id,product_id,purchase_date,page_views,time_spent,category,price,ratings,total_page_views,average_time_spent,total_purchases,recency_of_purchase,average_rating_purchased,average_price_purchased,purchase_month
0,1,101,2023-01-01,25,120,Electronics,500,4.5,500,120.0,9,360,4.040000,471.700000,1
1,1,105,2023-01-05,25,120,Electronics,800,4.8,500,120.0,9,356,4.040000,471.700000,1
2,2,102,2023-01-02,20,90,Clothing,50,3.8,160,90.0,5,359,3.937500,404.625000,1
3,3,103,2023-01-03,30,150,Home & Kitchen,200,4.2,240,150.0,5,358,3.712500,496.250000,1
4,4,104,2023-01-04,15,80,Beauty,30,4.0,240,80.0,6,357,4.043750,282.125000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,27,104,2023-01-18 00:00:00,22,92,Beauty,30,4.0,264,92.0,9,343,3.925000,480.416667,1
1002,49,105,2023-09-17 00:00:00,16,127,Electronics,800,4.8,160,127.0,8,101,3.980000,476.700000,9
1003,72,102,2023-05-07 00:00:00,17,116,Clothing,50,3.8,204,116.0,7,234,3.783333,243.583333,5
1004,55,107,2023-09-22 00:00:00,22,120,Books,783,3.8,220,120.0,7,96,3.830000,448.100000,9


In [39]:
df.dtypes

customer_id                   int64
product_id                    int64
purchase_date                object
page_views                    int64
time_spent                    int64
category                     object
price                         int64
ratings                     float64
total_page_views              int64
average_time_spent          float64
total_purchases               int64
recency_of_purchase           int64
average_rating_purchased    float64
average_price_purchased     float64
purchase_month                int64
dtype: object

In [67]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score

def predict_product(latest_customer_id):

        # Preprocess data (Replace with actual preprocessing logic)
        features = df[['page_views', 'time_spent', 'price', 'ratings', 'recency_of_purchase', 'purchase_month']]
        target = df['product_id']

        # Splitting the dataset into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

        # Train the Random Forest model
        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

        # Convert 'purchase_date' to datetime if it's not already
        df['purchase_date'] = pd.to_datetime(df['purchase_date'])

        # Sort the DataFrame by 'purchase_date' in descending order (latest first)
        terra_df_sorted = df.sort_values(by='purchase_date', ascending=False)

        # Fetch latest customer's data for prediction
        latest_customer_data = terra_df_sorted[terra_df_sorted['customer_id'] == latest_customer_id].iloc[0]
        latest_features = latest_customer_data[['page_views', 'time_spent', 'price', 'ratings', 'recency_of_purchase', 'purchase_month']]

        # Predict the product
        predicted_product_id = model.predict([latest_features])[0]
        return predicted_product_id

In [71]:
predict_product(4)

C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


101

**The use of a RandomForestClassifier in the model for predicting the next product purchase can be attributed to several inherent advantages of this algorithm**, particularly for classification tasks like yours. Here's why RandomForestClassifier is a suitable choice:

- **Handling of Both Categorical and Numerical Data:** RandomForest can naturally handle a mix of different feature types – in your case, both numerical (like page_views, time_spent, price, ratings, recency_of_purchase) and categorical data (purchase_month).

- **Robustness to Overfitting:** RandomForest, being an ensemble method that builds multiple decision trees and merges their outcomes, tends to be more robust against overfitting compared to individual decision trees. This is especially useful when you have a complex dataset.

- **Ability to Handle Non-Linear Relationships:** RandomForest can capture non-linear relationships between features and the target variable, which might be present in your dataset.

- **No Need for Feature Scaling:** RandomForest does not require feature scaling (like standardization or normalization) to be effective, which simplifies the preprocessing steps.

- **Handling Missing Values:** While your script does not explicitly handle missing values, RandomForest can handle them internally to some extent, which can be useful in many real-world datasets.

- **Feature Importance:** RandomForest can provide insights into the importance of different features in predicting the target variable, which can be valuable for understanding your model and the underlying data.

- **Good Performance with Default Parameters:** RandomForest often provides reasonably good results with default parameter settings, making it a good starting point for model development.

- **Versatility:** It is a versatile algorithm that can be a good first choice for many classification problems and usually performs well compared to other algorithms without extensive hyperparameter tuning.

**Why Not Other Algorithms?**

Linear Models (like Logistic Regression) might not perform well if the relationship between features and target variable is non-linear or complex.

SVM (Support Vector Machine) can be effective for classification tasks, but it may require careful tuning of hyperparameters and feature scaling.

Neural Networks could be an alternative, but they typically require more data, more computational resources, and more effort in tuning.

**In summary,** RandomForest provides a good balance between performance and ease of use, making it a suitable choice for many classification tasks, including yours. However, it's always a good idea to experiment with different models and compare their performance to find the best one for your specific dataset and problem.